# AgloK23 Trading System - System Overview

This notebook provides an overview of the AgloK23 quantitative trading system and demonstrates how to interact with its various components.


In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Add the src directory to Python path
sys.path.append('../src')

# Import AgloK23 modules
from config.settings import Settings
from core.data.models import MarketData, OHLCV
from core.features.engine import FeatureEngine
from core.models.manager import ModelManager

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("AgloK23 Trading System - Jupyter Environment Ready")

## 1. Configuration and Settings

Let's start by loading the system configuration:

In [ ]:
# Load system settings
settings = Settings()

print("System Configuration:")
print(f"Environment: {settings.ENVIRONMENT}")
print(f"Log Level: {settings.LOG_LEVEL}")
print(f"Redis Host: {settings.REDIS_HOST}")
print(f"Postgres Host: {settings.POSTGRES_HOST}")
print(f"Paper Trading: {settings.ENABLE_PAPER_TRADING}")
print(f"Default Portfolio Value: ${settings.DEFAULT_PORTFOLIO_VALUE:,.2f}")

## 2. Feature Engineering Demo

Let's demonstrate the feature engineering capabilities:

In [ ]:
# Create sample market data
dates = pd.date_range(start='2024-01-01', end='2024-01-31', freq='H')
np.random.seed(42)

# Generate synthetic OHLCV data
base_price = 50000
returns = np.random.normal(0, 0.02, len(dates))
prices = base_price * np.cumprod(1 + returns)

sample_data = []
for i, (date, price) in enumerate(zip(dates, prices)):
    high = price * (1 + abs(np.random.normal(0, 0.005)))
    low = price * (1 - abs(np.random.normal(0, 0.005)))
    volume = np.random.uniform(100, 1000)
    
    ohlcv = OHLCV(
        timestamp=date,
        open=prices[i-1] if i > 0 else price,
        high=high,
        low=low,
        close=price,
        volume=volume
    )
    sample_data.append(ohlcv)

print(f"Generated {len(sample_data)} OHLCV data points")

# Convert to DataFrame for visualization
df = pd.DataFrame([
    {
        'timestamp': d.timestamp,
        'open': d.open,
        'high': d.high,
        'low': d.low,
        'close': d.close,
        'volume': d.volume
    } for d in sample_data
])

df.set_index('timestamp', inplace=True)
print("\nSample data:")
print(df.head())

In [ ]:
# Visualize the sample data
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Price chart
axes[0].plot(df.index, df['close'], label='Close Price', linewidth=1.5)
axes[0].fill_between(df.index, df['low'], df['high'], alpha=0.3, label='High-Low Range')
axes[0].set_title('BTC/USD Price Chart (Sample Data)')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Volume chart
axes[1].bar(df.index, df['volume'], alpha=0.7, color='orange', width=0.02)
axes[1].set_title('Trading Volume')
axes[1].set_ylabel('Volume')
axes[1].set_xlabel('Date')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Technical Indicators Analysis

Let's compute some technical indicators:

In [ ]:
# Calculate moving averages
df['sma_20'] = df['close'].rolling(window=20).mean()
df['sma_50'] = df['close'].rolling(window=50).mean()
df['ema_20'] = df['close'].ewm(span=20).mean()

# Calculate RSI
def calculate_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df['rsi'] = calculate_rsi(df['close'])

# Calculate Bollinger Bands
df['bb_middle'] = df['close'].rolling(window=20).mean()
df['bb_std'] = df['close'].rolling(window=20).std()
df['bb_upper'] = df['bb_middle'] + (df['bb_std'] * 2)
df['bb_lower'] = df['bb_middle'] - (df['bb_std'] * 2)

print("Technical indicators calculated:")
print(df[['close', 'sma_20', 'sma_50', 'ema_20', 'rsi']].tail())

In [ ]:
# Visualize technical indicators
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Price and moving averages
axes[0].plot(df.index, df['close'], label='Close Price', linewidth=2)
axes[0].plot(df.index, df['sma_20'], label='SMA 20', alpha=0.8)
axes[0].plot(df.index, df['sma_50'], label='SMA 50', alpha=0.8)
axes[0].plot(df.index, df['ema_20'], label='EMA 20', alpha=0.8)
axes[0].fill_between(df.index, df['bb_lower'], df['bb_upper'], alpha=0.2, label='Bollinger Bands')
axes[0].set_title('Price Chart with Moving Averages and Bollinger Bands')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(df.index, df['rsi'], label='RSI', color='purple', linewidth=2)
axes[1].axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought (70)')
axes[1].axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold (30)')
axes[1].set_title('Relative Strength Index (RSI)')
axes[1].set_ylabel('RSI')
axes[1].set_ylim(0, 100)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Volume
axes[2].bar(df.index, df['volume'], alpha=0.7, color='orange', width=0.02)
axes[2].set_title('Trading Volume')
axes[2].set_ylabel('Volume')
axes[2].set_xlabel('Date')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Feature Correlation Analysis

Let's analyze correlations between different features:

In [ ]:
# Calculate returns and volatility
df['returns'] = df['close'].pct_change()
df['volatility'] = df['returns'].rolling(window=20).std()
df['price_change'] = df['close'].diff()

# Select features for correlation analysis
feature_cols = ['returns', 'volatility', 'rsi', 'volume', 'price_change']
correlation_data = df[feature_cols].dropna()

# Calculate correlation matrix
correlation_matrix = correlation_data.corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

print("\nCorrelation Matrix:")
print(correlation_matrix.round(3))

## 5. Risk Metrics Analysis

Let's calculate some basic risk metrics:

In [ ]:
# Calculate risk metrics
returns = df['returns'].dropna()

# Basic statistics
mean_return = returns.mean()
std_return = returns.std()
sharpe_ratio = mean_return / std_return * np.sqrt(24 * 365)  # Annualized for hourly data

# Value at Risk (VaR) - 95% confidence
var_95 = np.percentile(returns, 5)
var_99 = np.percentile(returns, 1)

# Maximum drawdown
cumulative_returns = (1 + returns).cumprod()
rolling_max = cumulative_returns.expanding().max()
drawdown = (cumulative_returns - rolling_max) / rolling_max
max_drawdown = drawdown.min()

print("Risk Metrics:")
print(f"Mean Return: {mean_return:.4f}")
print(f"Return Volatility: {std_return:.4f}")
print(f"Sharpe Ratio (Annualized): {sharpe_ratio:.2f}")
print(f"VaR (95%): {var_95:.4f}")
print(f"VaR (99%): {var_99:.4f}")
print(f"Maximum Drawdown: {max_drawdown:.4f}")

In [ ]:
# Visualize risk metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Returns distribution
axes[0, 0].hist(returns, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].axvline(var_95, color='red', linestyle='--', label=f'VaR 95%: {var_95:.4f}')
axes[0, 0].axvline(var_99, color='darkred', linestyle='--', label=f'VaR 99%: {var_99:.4f}')
axes[0, 0].set_title('Returns Distribution')
axes[0, 0].set_xlabel('Returns')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Cumulative returns
axes[0, 1].plot(cumulative_returns.index, cumulative_returns, label='Cumulative Returns')
axes[0, 1].plot(rolling_max.index, rolling_max, label='Rolling Maximum', alpha=0.7)
axes[0, 1].set_title('Cumulative Returns')
axes[0, 1].set_ylabel('Cumulative Return')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Drawdown
axes[1, 0].fill_between(drawdown.index, drawdown, 0, alpha=0.7, color='red')
axes[1, 0].set_title('Drawdown')
axes[1, 0].set_ylabel('Drawdown')
axes[1, 0].set_xlabel('Date')
axes[1, 0].grid(True, alpha=0.3)

# Rolling volatility
rolling_vol = returns.rolling(window=24).std()  # 24-hour rolling volatility
axes[1, 1].plot(rolling_vol.index, rolling_vol, color='orange')
axes[1, 1].set_title('Rolling Volatility (24h)')
axes[1, 1].set_ylabel('Volatility')
axes[1, 1].set_xlabel('Date')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. System Health Check

Let's check the status of various system components:

In [ ]:
import requests
import redis
import psycopg2
from datetime import datetime

def check_service_health():
    health_status = {}
    
    # Check Redis
    try:
        r = redis.Redis(host='localhost', port=6379, db=0)
        r.ping()
        health_status['Redis'] = '✅ Online'
    except Exception as e:
        health_status['Redis'] = f'❌ Offline: {str(e)}'
    
    # Check PostgreSQL
    try:
        conn = psycopg2.connect(
            host='localhost',
            port=5432,
            database='algok23',
            user='postgres',
            password='password'
        )
        conn.close()
        health_status['PostgreSQL'] = '✅ Online'
    except Exception as e:
        health_status['PostgreSQL'] = f'❌ Offline: {str(e)}'
    
    # Check TimescaleDB
    try:
        conn = psycopg2.connect(
            host='localhost',
            port=5433,
            database='algok23_timeseries',
            user='postgres',
            password='password'
        )
        conn.close()
        health_status['TimescaleDB'] = '✅ Online'
    except Exception as e:
        health_status['TimescaleDB'] = f'❌ Offline: {str(e)}'
    
    # Check MLflow
    try:
        response = requests.get('http://localhost:5000/health', timeout=5)
        if response.status_code == 200:
            health_status['MLflow'] = '✅ Online'
        else:
            health_status['MLflow'] = f'❌ HTTP {response.status_code}'
    except Exception as e:
        health_status['MLflow'] = f'❌ Offline: {str(e)}'
    
    # Check Prometheus
    try:
        response = requests.get('http://localhost:9090/-/healthy', timeout=5)
        if response.status_code == 200:
            health_status['Prometheus'] = '✅ Online'
        else:
            health_status['Prometheus'] = f'❌ HTTP {response.status_code}'
    except Exception as e:
        health_status['Prometheus'] = f'❌ Offline: {str(e)}'
    
    # Check Grafana
    try:
        response = requests.get('http://localhost:3000/api/health', timeout=5)
        if response.status_code == 200:
            health_status['Grafana'] = '✅ Online'
        else:
            health_status['Grafana'] = f'❌ HTTP {response.status_code}'
    except Exception as e:
        health_status['Grafana'] = f'❌ Offline: {str(e)}'
    
    return health_status

print(f"System Health Check - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 50)

health = check_service_health()
for service, status in health.items():
    print(f"{service:15} {status}")

print("=" * 50)
online_services = sum(1 for status in health.values() if '✅' in status)
total_services = len(health)
print(f"Services Online: {online_services}/{total_services}")

## 7. Quick Start Commands

Here are some useful commands to get started with the AgloK23 system:

In [ ]:
print("AgloK23 Quick Start Commands:")
print("\n📊 System Monitoring:")
print("  - Grafana Dashboard: http://localhost:3000")
print("  - Prometheus Metrics: http://localhost:9090")
print("  - MLflow Tracking: http://localhost:5000")

print("\n🐳 Docker Commands:")
print("  - Start services: docker-compose up -d")
print("  - Stop services: docker-compose down")
print("  - View logs: docker-compose logs -f")
print("  - Rebuild: docker-compose build --no-cache")

print("\n🔧 Development:")
print("  - Run tests: pytest tests/")
print("  - Format code: black src/")
print("  - Type check: mypy src/")
print("  - Start app: uvicorn src.main:app --reload")

print("\n📈 Trading Operations:")
print("  - Check positions: curl http://localhost:8000/api/positions")
print("  - Health check: curl http://localhost:8000/health")
print("  - System metrics: curl http://localhost:8000/metrics")

print("\n💡 Next Steps:")
print("  1. Configure your .env file with API keys")
print("  2. Start the data ingestion services")
print("  3. Train your first ML models")
print("  4. Deploy trading strategies")
print("  5. Monitor performance via Grafana")

## Conclusion

This notebook provides a basic overview of the AgloK23 trading system. The system includes:

- **Data Pipeline**: Real-time and historical data ingestion from multiple sources
- **Feature Engineering**: 100+ technical indicators and market features
- **Machine Learning**: Multiple model types with ensemble capabilities
- **Risk Management**: Portfolio risk metrics and position sizing
- **Execution**: Smart order routing and multi-venue trading
- **Monitoring**: Comprehensive system health and performance tracking

To get started, make sure all services are running via Docker Compose and explore the various components through their respective interfaces.